# Comparativo de desemenho entre Redes Neurais Convolucionais padrões: Acurácia, Loss, Matriz de Confusão e Relatório de Classificação

## Importação das bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import shutil
import matplotlib.pyplot as plt
import pylab as pl
from keras import backend as K
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import keras
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization

## Diretórios do dataset

In [ ]:
train = '../input/tccovid3/tccovid/train'
val = '../input/tccovid3/tccovid/test'

## Verificação do número de imagens - Treinamento e Teste

In [ ]:
# Total number of training images
num_of_train_samples = 0
for train_dataset in os.listdir(train):
    in_folder = train + "/" + train_dataset 
    in_folder_list = os.listdir(in_folder)
    num_of_train_samples = num_of_train_samples + len(in_folder_list)
print("Número de Amostras para Treinamento: ",num_of_train_samples)

# Total number of validation images
num_of_validation_samples = 0
for validation_dataset in os.listdir(val):
    in_folder_val = val + "/" + validation_dataset
    in_folder_val_list = os.listdir(in_folder_val)
    num_of_validation_samples = num_of_validation_samples + len(in_folder_val_list)
print("Número de Amostras para Teste: ", num_of_validation_samples)

## Gerador de Imagens: Aplicação da Técnica de Aumento de Dados

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255)

## Definindo os hiperparâmetros, resolução da imagem e classes do dataset

In [ ]:
batch_size = 16
target_names = ['COVID', 'PNEUMONIA', 'NORMAL']

# Defining image width and height respectively
img_rows = 224
img_cols = 224

## Definindo as amotras de Treinamento e Teste

In [ ]:
train_generator = train_datagen.flow_from_directory(train,
                                                    target_size = (img_rows, img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(val,
                                                        target_size = (img_rows, img_cols),
                                                        batch_size = batch_size,
                                                        shuffle = False, class_mode='categorical')

# Definição da Função para gerar a Matriz de Confusão

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de Confusão Normalizada")
    else:
        print('Matriz de Confusão Não Normalizada')
    
    import itertools
    print(cm)
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')
    plt.tight_layout()

# ResNet50

## Definindo a quantidade de épocas, passos por época e  batch size

In [ ]:
steps_per_epoch = num_of_train_samples // batch_size
batch_size = 16
epoch = 15
print("Passos por época: ",steps_per_epoch)
learning_rate = 1e-5

## Importando e Criando o modelo

In [ ]:
from keras.applications import ResNet50
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, 
                   pooling='max', 
                   weights='imagenet'))
resnet_model.add(BatchNormalization())
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation = "relu"))
resnet_model.add(Dense(3, activation='softmax'))


# Summary: to find the number of parameters
resnet_model.layers[0].trainable=False
resnet_model.summary()

#sgd = optimizers.SGD(lr=learning_rate, decay=0.0001,momentum = 0.0,nesterov=False)

## Compilando o modelo

In [ ]:
resnet_model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=["accuracy"])

## Treinando e Validando o modelo

In [ ]:
resnet_training = resnet_model.fit(train_generator,
                               steps_per_epoch = steps_per_epoch,
                               epochs = epoch,
                               validation_data = validation_generator,
                               validation_steps = num_of_validation_samples)

## Acurácia e Loss de Treinamento e Teste

In [ ]:
training_accuracy_resnet      = resnet_training.history['accuracy'][-1]
training_loss_resnet          = resnet_training.history['loss'][-1]
validation_accuracy_resnet    = resnet_training.history['val_accuracy'][-1]
validation_loss_resnet        = resnet_training.history['val_loss'][-1]
print("Training Accuracy ResNet   :", training_accuracy_resnet)
print("Training Loss ResNet       :", training_loss_resnet)
print("Validation Accuracy ResNet :", validation_accuracy_resnet)
print("Validation Loss ResNet     :", validation_loss_resnet)

## Gerando a Matriz de Confusão e o Relatório de classificação

In [ ]:
Y_pred_res = resnet_model.predict_generator(validation_generator, num_of_validation_samples // batch_size+1)
y_pred_resnet = np.argmax(Y_pred_res, axis=1)
print('Matriz de Confusão')
conf_matrix_res = confusion_matrix(validation_generator.classes, y_pred_resnet)
cm_res = np.array2string(conf_matrix_res)
print(conf_matrix_res)
print("=============================================================================================")
print('Relatório de Classificação')
class_rep_res = classification_report(validation_generator.classes, y_pred_resnet, target_names=target_names)
print(class_rep_res)

## Matriz de Confusão

In [ ]:
print('Matriz de Confusão')
conf_matrix_res = confusion_matrix(validation_generator.classes, y_pred_resnet)
plot_confusion_matrix(conf_matrix_res, classes=target_names, title='Matriz de Confusão - Resnet50')

# Resnet com Fine Tune

## Importando e Criando o modelo

In [ ]:
from keras.applications import ResNet50
conv_base = ResNet50(weights='imagenet',
include_top=False,
input_shape=(224, 224, 3))


model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

print(model.summary())

## Tornando o último bloco da Resnet50 como treinável:

In [ ]:
for layer in conv_base.layers[:165]:
   layer.trainable = False
for layer in conv_base.layers[165:]:
   layer.trainable = True

## Compilando o modelo

In [ ]:
model.compile(optimizer=optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

## Treinando e Validando o modelo

In [ ]:
resnet_training2 = model.fit(train_generator,
                               steps_per_epoch = steps_per_epoch,
                               epochs = epoch,
                               validation_data = validation_generator,
                               validation_steps = num_of_validation_samples)

## Acurácia e Loss de Treinamento e Teste

In [ ]:
training_accuracy_resnet2      = resnet_training2.history['accuracy'][-1]
training_loss_resnet2          = resnet_training2.history['loss'][-1]
validation_accuracy_resnet2    = resnet_training2.history['val_accuracy'][-1]
validation_loss_resnet2        = resnet_training2.history['val_loss'][-1]
print("Training Accuracy ResNet   :", training_accuracy_resnet2 )
print("Training Loss ResNet       :", training_loss_resnet2)
print("Validation Accuracy ResNet :", validation_accuracy_resnet2)
print("Validation Loss ResNet     :", validation_loss_resnet2)

## Gerando a Matriz de Confusão e o Relatório de classificação

In [ ]:
Y_pred_res = model.predict_generator(validation_generator, num_of_validation_samples // batch_size+1)
y_pred_res = np.argmax(Y_pred_res, axis=1)
print('Matriz de Confusão')
conf_matrix_res = confusion_matrix(validation_generator.classes, y_pred_res)
cm_res = np.array2string(conf_matrix_res)
print(conf_matrix_res)
print("=============================================================================================")
print('Relatório de classificação')
class_rep_res = classification_report(validation_generator.classes, y_pred_res, target_names=target_names)
print(class_rep_res)


## Matriz de Confusão

In [ ]:
print('Matriz de Confusão')
conf_matrix_res = confusion_matrix(validation_generator.classes, y_pred_res)
plot_confusion_matrix(conf_matrix_res, classes=target_names, title='Matriz de Confusão - Resnet50 com Fine Tune')

# Inception

## Definindo a quantidade de épocas, passos por época e Batch Size

In [ ]:
epoch_inc = 15
batch_size_inc = 32

steps_per_epoch_inc = num_of_train_samples // batch_size_inc
print("Steps per epoch: ",steps_per_epoch_inc)

## Importando e Criando o modelo

In [ ]:
from keras.applications import InceptionV3
inc_model = Sequential()
inc_model.add(InceptionV3(include_top=False, 
                      pooling='max',
                      weights='imagenet'))
inc_model.add(Dense(3, activation="softmax"))

# Summary: to find the number of parameters
inc_model.layers[0].trainable=False
inc_model.summary()

## Compilando o modelo

In [ ]:
inc_model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=["accuracy"])

## Treinando e Validando o modelo

In [ ]:
inc_training = inc_model.fit_generator(train_generator,
                                       steps_per_epoch = steps_per_epoch_inc,
                                       epochs = epoch_inc,
                                       validation_data = validation_generator,
                                       validation_steps = num_of_validation_samples)

## Acurácia e Loss de Treinamento e Teste

In [ ]:
training_accuracy_inc      = inc_training.history['accuracy'][-1]
training_loss_inc          = inc_training.history['loss'][-1]
validation_accuracy_inc    = inc_training.history['val_accuracy'][-1]
validation_loss_inc        = inc_training.history['val_loss'][-1]
print("Training Accuracy Inception   :", training_accuracy_inc )
print("Training Loss Inception       :", training_loss_inc)
print("Validation Accuracy Inception :", validation_accuracy_inc)
print("Validation Loss Inception     :", validation_loss_inc)

## Gerando a Matriz de Confusão e o Relatório de classificação

In [ ]:
Y_pred_inc = inc_model.predict_generator(validation_generator, num_of_validation_samples // batch_size+1)
y_pred_inc = np.argmax(Y_pred_inc, axis=1)
print('Matriz de Confusão')
conf_matrix_inc = confusion_matrix(validation_generator.classes, y_pred_inc)
cm_inc = np.array2string(conf_matrix_inc)
print(conf_matrix_inc)
print("=============================================================================================")
print('Relatório de classificação')
class_rep_inc = classification_report(validation_generator.classes, y_pred_inc, target_names=target_names)
print(class_rep_inc)

## Matriz de Confusão

In [ ]:
print('Matriz de Confusão')
conf_matrix_inc = confusion_matrix(validation_generator.classes, y_pred_inc)
plot_confusion_matrix(conf_matrix_inc, classes=target_names, title='Matriz de Confusão - Inception')

# VGG16

## Definindo a quantidade de épocas e Batch Size

In [ ]:
epoch_vgg = 15
learning_rate_vgg = 0.001
batch_size_vgg = 64
steps_per_epoch_vgg = num_of_train_samples // batch_size_vgg
print("Passos por época: ",steps_per_epoch_vgg)

## Importando e Criando o modelo

In [ ]:
from keras.applications import vgg16
vgg_model=Sequential()
vgg_model.add(vgg16.VGG16(include_top = False, pooling = 'max', weights = 'imagenet'))
vgg_model.add(Dense(3, activation="softmax"))

    # Summary: to find the number of parameters
vgg_model.layers[0].trainable=False
vgg_model.summary() 

## Compilando o modelo

In [ ]:
vgg_model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=1e-5),
                  metrics=["accuracy"])

## Treinando e Validando o modelo

In [ ]:
vgg_training = vgg_model.fit_generator(train_generator,
                                       steps_per_epoch = steps_per_epoch_vgg,
                                       epochs = epoch_vgg,
                                       validation_data = validation_generator,
                                       validation_steps = num_of_validation_samples)

## Acurácia e Loss de Treinamento e Teste

In [ ]:
training_accuracy_vgg      = vgg_training.history['accuracy'][-1]
training_loss_vgg          = vgg_training.history['loss'][-1]
validation_accuracy_vgg    = vgg_training.history['val_accuracy'][-1]
validation_loss_vgg        = vgg_training.history['val_loss'][-1]
print("Training Accuracy VGG    :", training_accuracy_vgg )
print("Training Loss VGG        :", training_loss_vgg)
print("Validation Accuracy VGG  :", validation_accuracy_vgg)
print("Validation Loss VGG      :", validation_loss_vgg)

## Gerando a Matriz de Confusão e o Relatório de classificação

In [ ]:
Y_pred_vgg = vgg_model.predict_generator(validation_generator, num_of_validation_samples // batch_size+1)
y_pred_vgg = np.argmax(Y_pred_vgg, axis=1)
print('Matriz de Confusão')
conf_matrix_vgg = confusion_matrix(validation_generator.classes, y_pred_vgg)
cm_vgg = np.array2string(conf_matrix_vgg)
print(conf_matrix_vgg)
print("=============================================================================================")
print('Relatório de classificação')
class_rep_vgg = classification_report(validation_generator.classes, y_pred_vgg, target_names=target_names)
print(class_rep_vgg)

## Matriz de Confusão

In [ ]:
print('Matriz de Confusão')
conf_matrix_vgg = confusion_matrix(validation_generator.classes, y_pred_vgg)
plot_confusion_matrix(conf_matrix_vgg, classes=target_names, title='Matriz de Confusão - VGG')

# Comparação das Acurácias e Loss de todos os modelos

In [ ]:

model_comp = pd.DataFrame({"Modelos": ['ResNet50', 'ResNet50 com Fine Tuning','Inception', 'VGG16'],
                           "Batch Size":[16,16,32,64],
                           "Epócas":[epoch,epoch,epoch_inc,epoch_vgg],
                           "Taxa de Aprendizado": [0.00001,0.00001,0.00001,0.00001],
                           "Passos por Época":[steps_per_epoch,steps_per_epoch,steps_per_epoch_inc,steps_per_epoch_vgg],
                           "Resolução da Imagem":['224*224','224*224','224*224', '224*224'],
                          "Acurácia no Treinamento": [training_accuracy_resnet,training_accuracy_resnet2,training_accuracy_inc,training_accuracy_vgg],
                          "Loss no Treinamento": [training_loss_resnet,training_loss_resnet2,training_loss_inc,training_loss_vgg],
                          "Acurácia no Teste": [validation_accuracy_resnet,validation_accuracy_resnet2,validation_accuracy_inc,validation_accuracy_vgg],
                          "Loss no Teste": [validation_loss_resnet,validation_loss_resnet2,validation_loss_inc,validation_loss_vgg],
                          })
model_comp

In [ ]:
plt.plot(resnet_training.history['loss'], label='Resnet Loss')
plt.plot(resnet_training2.history['loss'], label='Resnet Fine Tune Loss')
plt.plot(inc_training.history['loss'], label='Inception Loss')
plt.plot(vgg_training.history['loss'], label='VGG Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
plt.plot(resnet_training.history['accuracy'], label='Resnet Accuracy')
plt.plot(resnet_training2.history['accuracy'], label='Resnet Fine Tune Accuracy')
plt.plot(inc_training.history['accuracy'], label='Inception Accuracy')
plt.plot(vgg_training.history['accuracy'], label='VGG Accuracy')
plt.legend()
plt.show()
plt.savefig('LossVal_Accuracy')